In [4]:
from SALib.sample import saltelli
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import random, math


In [11]:
path_survival = './storage/intermediante/kaz_survival_rates_f.csv'
path_population = './storage/intermediante/kaz_population_both.csv'
path_bgr = './storage/intermediante/kaz_boys_girls_ratio.csv'

In [12]:
cols = [
    'Index', 'Variant', 'Area', 'Notes',
    'Country_code', 'Reference_date',
    '0 - 4', '5 - 9', '10 - 14', '15 - 19',
    '20 - 24', '25 - 29', '30 - 34', '35 - 39',
    '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64',
    '65 - 69', '70 - 74', '75 - 79', '80 - 84', '85 - 89', '90 - 94',
    '95 - 99', '100+'
]
cols_ext_period = [ 'Sex',
    '0 - 4', '5 - 9', '10 - 14', '15 - 19',
    '20 - 24', '25 - 29', '30 - 34', '35 - 39',
    '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64',
    '65 - 69', '70 - 74', '75 - 79', '80 - 84'
    ,'85 - 89', '90 - 94', '95 - 99', '100+'
]
cols_period = [
    '0 - 4', '5 - 9', '10 - 14', '15 - 19',
    '20 - 24', '25 - 29', '30 - 34', '35 - 39',
    '40 - 44', '45 - 49', '50 - 54', '55 - 59', '60 - 64',
    '65 - 69', '70 - 74', '75 - 79', '80 - 84'
    ,'85 - 89', '90 - 94', '95 - 99', '100+'
]

In [13]:
r1 = pd.read_csv(path_survival, index_col='index')
r1

,0 - 4,5 - 9,10 - 14,15 - 19,20 - 24,25 - 29,30 - 34,35 - 39,40 - 44,45 - 49,...,55 - 59,60 - 64,65 - 69,70 - 74,75 - 79,80 - 84,85 - 89,90 - 94,95 - 99,100+
index,,,,,,,,,,,,,,,,,,,,,
0,1.450869,1.90301,1.280586,2.788492,1.770657,2.922022,3.540247,2.188188,2.938326,2.850477,...,2.197028,1.665471,3.146063,1.97562,3.622302,1.899271,38.87,21.434,2.955,0.476


In [14]:
r2 = pd.read_csv(path_population, index_col='index')
r2

,0 - 4,5 - 9,10 - 14,15 - 19,20 - 24,25 - 29,30 - 34,35 - 39,40 - 44,45 - 49,50 - 54,55 - 59,60 - 64,65 - 69,70 - 74,75 - 79,80 - 84
index,,,,,,,,,,,,,,,,,
0,1.038667,1.032959,1.03413,1.040998,1.030738,0.997853,0.973107,0.93607,0.896203,0.844101,0.793848,0.746739,0.692477,0.633735,0.567595,0.495103,0.441721


In [7]:
r3 = pd.read_csv(path_bgr, index_col='index')
r3

,0 - 4,5 - 9,10 - 14,15 - 19,20 - 24,25 - 29,30 - 34,35 - 39,40 - 44,45 - 49,50 - 54,55 - 59,60 - 64,65 - 69,70 - 74,75 - 79,80 - 84
index,,,,,,,,,,,,,,,,,
0,1.038667,1.032959,1.03413,1.040998,1.030738,0.997853,0.973107,0.93607,0.896203,0.844101,0.793848,0.746739,0.692477,0.633735,0.567595,0.495103,0.441721


In [ ]:
def Model(x1,x2,x3):
    return x1+x2+2*x3#+5*random.random()

def Evaluate(param_values):
    Y = []
    for params in param_values:
        x1,x2,x3 = params
        res = Model(x1,x2,x3)                
        Y.append(res)
    return np.array(Y)

In [ ]:
# Define the model inputs
problem = {
    'num_vars': 3,
    'names': ['x1', 'x2', 'x3'],
    'bounds': [[-1, 1],
               [-1, 1],
               [-1, 1]]
}

# 1 task

Perform a **sensitivity analysis** for a demographic model with respect to a set of parameters: fertility rate, boys/girls ratio, «survival» rate for different age groups (not all can be taken). Model output: number of inhabitants for a given year. Test on the final forecast values ​​for 10, 20, 50, 100 years.

# 2 task

Define ranges of model parameter values ​​from data for previous periods (1950-2000)

# 3 task

Based on all ranges of parameter values, perform an **uncertainty analysis** in the form of a graph with confidence intervals of the results. The values ​​between the boundaries can be considered evenly distributed.